### Building Langgraph graphs using the Agent Executor
In this notebook, we will go over building an agent executor using langgraph from scratch

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_experimental.tools.python.tool import PythonREPLTool

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents.agent import AgentExecutor


tools = [TavilySearchResults(max_results=1), PythonREPLTool()]
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent_runnable, tools=tools)


In [31]:
agent_executor.invoke({"input": "what is the weather in sf?"})

{'input': 'what is the weather in sf?',
 'output': ' [\n  {\n    "name": "tavily_search_results_json",\n    "arguments": {\n      "query": "weather in sf"\n    }\n  }\n]'}

##### Define the graph state
We now define the graph state. The state for the tranditional langchain agent has a few attributes: 

- input : This is the input string representing the main ask from the user, passed in as input
- chat_history: This is any previous conversation messages, also passed in as input
- intermediate_steps: This is a list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent
- agent_outcome: This is the response from the agent, either an `AgentAction` or `AgentFinsh`. The AgentExecutor should finish when this is an agent finish, ohterwise it should call the requested tool

In [10]:
from typing import TypedDict,Annotated, Union, Sequence, Tuple
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

In [32]:
class AgentState(TypedDict):
    input: Annotated[str, operator.setitem]
    chat_history: Annotated[Sequence[BaseMessage], operator.add]
    agent_outcome: Annotated[Union[AgentAction, AgentFinish, None], operator.setitem]
    intermediate_steps: Annotated[Sequence[Tuple[AgentAction, str]], operator.add]

#### Define the nodes
We now need to define a few different ndoes in our graph. In `langgraph`, a node cna be either a function or a runnable. There are two main nodes we need for thisl 
- The agent: responsible for deciding what (if any) action to take
- A function to invoke tools: if the agent decides to taks an action, this node will then execute the action 

We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of the node, one of several paths may be taken. The path that is taken is not known until that node is run (the LLM decides).

- Conditional Edge: after the agent is called, we should either: a. if the agent said to take an action, then the functio nto invoke tools shoudl be called b. If the agent said that it is finished, then it should finish
- Normal Edge: after the tools are invokved, it should always go back to the agent to decide what to do next


In [33]:
from langgraph.prebuilt import ToolExecutor
from langchain_core.agents import AgentFinish


tool_executor = ToolExecutor(tools)


def run_agent(state: AgentState):
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}

def execute_tools(state: AgentState):
    agent_action = state["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_contine(state: AgentState):
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    
    else:
        return "continue"

### Define the graph
We can now put everything together into a graph

In [34]:
from langgraph.graph import StateGraph, END


workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.set_entry_point("agent")

workflow.add_conditional_edges("agent", should_contine, {"continue": "action", "end": END})
workflow.add_edge("action", "agent")

app = workflow.compile()

In [35]:
inputs = {
    "input": "what is the weather in sf?", "chat_history": [], "intermediate_steps": []
}

In [36]:
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node {key}")
        print("----")
        print(value)
    print("----")

output from node agent
----
{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----
output from node __end__
----
{'input': 'what is the weather in sf?', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''), 'intermediate_steps': []}
----
